In [1]:
import os
import nltk
from google.colab import drive

In [2]:
!pip install transformers seqeval datasets
!apt install git-lfs
#!python3 -m spacy download en_core_web_lg
!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.1.0/en_core_web_lg-3.1.0-py3-none-any.whl

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
     |████████████████████████████████| 777.1 MB 16 kB/s 


In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
mount_folder = os.path.join('/content', 'drive')
project_path = os.path.join(mount_folder, 'MyDrive', 'projects', 'product-absa')
drive.mount(mount_folder)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd {project_path}

/content/drive/MyDrive/projects/product-absa


In [6]:
from src.models.aspect_based_sentiment_analysis_mgr import AspectSentimentManager
aspect_sentiment_manager = AspectSentimentManager()

## Perform aspect extraction and sentiment analysis small batch

In [7]:
    review_batch = [
                    'The food we had yesterday was delicious. The breakfast was delivered late',
                    'My time in Italy was very enjoyable',
                    'I found the meal to be tasty',
                    'The internet connection was terribly slow.',
                    'Our experience was suboptimal',
                    'It was a surprise, the screen is not bad',
                    'Not sure if the movie sound was the most suitable one'
                    ]


In [8]:
df_aspects_sentiment = aspect_sentiment_manager.predict_batch(text_reviews=review_batch, flatten=False)
df_aspects_sentiment_flatten = aspect_sentiment_manager.predict_batch(text_reviews=review_batch, flatten=True)

7it [00:00, 11.74it/s]
7it [00:00, 11.72it/s]


In [9]:
df_aspects_sentiment

,aspect,modifier,rule,aspect_idx,aspect_i,aspect_sentence,id_review,review,sentiment,sentiment_prob,aspects_aux,polarity_aux,polarity_prob_aux
0,food,delicious,r3,4,1,"(The, food, we, had, yesterday, was, delicious)",0,The food we had yesterday was delicious. The b...,Positive,0.997638,"[food, breakfast]","[Positive, Positive]","[0.997616171836853, 0.9975097179412842]"
1,breakfast,late,r4,4,1,"(The, breakfast, was, delivered, late)",0,The food we had yesterday was delicious. The b...,neutral,0.598951,"[food, breakfast]","[Positive, Positive]","[0.997616171836853, 0.9975097179412842]"
0,time,enjoyable,r3,3,1,"(My, time, in, Italy, was, very, enjoyable)",1,My time in Italy was very enjoyable,Positive,0.997235,[None],[Positive],[0.997235119342804]
0,meal,tasty,r3,12,3,"(I, found, the, meal, to, be, tasty)",2,I found the meal to be tasty,Positive,0.998121,[meal],[Positive],[0.9981209635734558]
0,connection,slow,r3,13,2,"(The, internet, connection, was, terribly, slow)",3,The internet connection was terribly slow.,Negative,0.903734,[internet connection],[Negative],[0.8881391882896423]
0,experience,suboptimal,r3,4,1,"(Our, experience, was, suboptimal)",4,Our experience was suboptimal,Negative,0.901462,[None],[Negative],[0.9014618992805481]
0,screen,not bad,r3,23,6,"(It, was, a, surprise, ,, the, screen, is, not...",5,"It was a surprise, the screen is not bad",Positive,0.949025,[screen],[Positive],[0.9490248560905457]
0,one,most suitable,r1,50,10,"(Not, sure, if, the, movie, sound, was, the, m...",6,Not sure if the movie sound was the most suita...,Positive,0.949499,[movie sound],[Positive],[0.9494986534118652]


In [10]:
df_aspects_sentiment_flatten

,id_review,review,aspects,polarity,aspects_aux,polarity_aux
0,0,The food we had yesterday was delicious. The b...,"[food, breakfast]","[Positive, neutral]","[food, breakfast]","[Positive, Positive]"
0,1,My time in Italy was very enjoyable,[time],[Positive],[None],[Positive]
0,2,I found the meal to be tasty,[meal],[Positive],[meal],[Positive]
0,3,The internet connection was terribly slow.,[connection],[Negative],[internet connection],[Negative]
0,4,Our experience was suboptimal,[experience],[Negative],[None],[Negative]
0,5,"It was a surprise, the screen is not bad",[screen],[Positive],[screen],[Positive]
0,6,Not sure if the movie sound was the most suita...,[one],[Positive],[movie sound],[Positive]


## Perform aspect extraction and sentiment analysis large batch

In [11]:
import pandas as pd
df = pd.read_csv('data/processed/reviews_dataset.csv',sep=';')
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,source,product_name,review_text,rating,polarity
0,0,google-play,Coloring book moana,A kid's excessive ads The types ads allowed ap...,-0.250,Negative
1,1,google-play,Coloring book moana,It bad :(,-0.725,Negative
2,2,google-play,Coloring book moana,like,0.000,Neutral
3,3,google-play,Coloring book moana,NaN,NaN,NaN
4,4,google-play,Coloring book moana,I love colors inspyering,0.500,Positive
...,...,...,...,...,...,...
1783161,1621951,lara-tripadvisor,78587,The hotel is right by Hwy 29 where most of the...,3.000,Positive
1783162,1621952,lara-tripadvisor,78587,I wanted to find a comfortable room that would...,5.000,Positive
1783163,1621953,lara-tripadvisor,78587,"In spite of its cheesy name, I had a great exp...",4.000,Positive
1783164,1621954,lara-tripadvisor,78587,We were newly married stayed here for 2 nights...,4.000,Positive


In [22]:
df = df.sample(n=100, random_state=1)
review_batch_large = df.review_text.to_list()

In [23]:
df_aspects_sentiment_flatten_large = aspect_sentiment_manager.predict_batch(text_reviews=review_batch_large, flatten=True)

100it [02:59,  1.79s/it]


In [24]:
df_aspects_sentiment_flatten_large

,id_review,review,aspects,polarity,aspects_aux,polarity_aux
0,0,Although the Red Rock is a ways from the strip...,"[(the, Red, Rock), None]","[Positive, Negative]","[rooms, atmosphere]","[Positive, Positive]"
0,1,I got a room here for the NYC marathon I was n...,"[things, water, shower, water, floor, room, pa...","[Positive, Positive, Negative, Negative, Negat...","[room, water]","[Positive, Negative]"
0,2,"This hotel has everything! Large, stylish and ...","[None, rooms, staff, pool, breakfast, Its, pla...","[Positive, Positive, Positive, Positive, Posit...","[rooms, staff, pool, breakfast]","[Positive, Positive, Positive, Positive]"
0,3,"Really poor service if it could be called so, ...","[service, rooms, showers, showers]","[Negative, Negative, Negative, Negative]","[service, WiFi]","[Negative, Negative]"
0,4,I stayed at the winchester in Dec 06 with my h...,"[deal, something, type, problems, shower, warm...","[Positive, Positive, Negative, Positive, Posit...",[price],[Positive]
...,...,...,...,...,...,...
0,95,I was celebrating a special weekend and to be ...,"[weekend, view, size, None, bar, side, staff, ...","[Positive, Positive, Positive, Negative, neutr...",[room],[Negative]
0,96,Fantastic Hotel drifting piling barge,"[barge, barge]","[Negative, Negative]",[Hotel],[Negative]
0,97,I found the Hotel Edison forty years ago and I...,"[charm, features, staff, None]","[Positive, Positive, Positive, Positive]",[None],[Positive]
0,98,"Uphill from Union Square, good budget hotel My...","[budget, price, hotel, hotels, strike, hotels,...","[Positive, Positive, Positive, Positive, Posit...",[hotel],[Positive]


In [20]:
df_aspects_sentiment_flatten_large.to_csv('df_aspects_sentiment_flatten_large.csv', sep=';', index=False)